In [2]:
import pandas as pd
import numpy as np
import neattext as nt
import neattext.functions as nfx

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [186]:
df = pd.read_csv("lda_topics.csv")
df.head()

,Topic #01,Topic #02,Topic #03,Topic #04,Topic #05,Topic #06,Topic #07,Topic #08,Topic #09,Topic #10
0,settlement,type,rate,answer,question,temperature,population,feature,Table,road
1,Using,earthquake,migration,cross_section,number,show,area,shown,show,edge_map
2,Suggest,land_use,population,volcano,name,rainfall,country,map_extract,Study,point
3,reason,Name,growth,complete,use,change,two,Describe,water,Tick_one
4,one,industry,Give,height,Insert,climate,year,river,position,Tick


In [196]:
df2 = pd.read_csv("data.csv")
df2.head()

,Paper_ID,Question_ID,Question_Type,Question_Text,Token_Text,Word_Count
0,0460_m17_qp_22.pdf,1,MAIN,"Study the map extract for Ballyvaghan, Ireland...",Study map extract Ballyvaghan Ireland scale 1:50,71
1,0460_m17_qp_22.pdf,(a),SUB,The map has blue grid lines which make square...,map blue grid line make square area land one g...,126
2,0460_m17_qp_22.pdf,10,MAIN,km2,NaN,4
3,0460_m17_qp_22.pdf,20,MAIN,km2,NaN,4
4,0460_m17_qp_22.pdf,50,MAIN,km2 100 km2,NaN,12


In [6]:
x = 'map_extract'
q = x.split('_')
print(q)

['map', 'extract']


In [197]:
# df.iloc[0:5, 1:11].values.flatten().tolist()
# df.values.flatten().tolist()
# important_tags_raw = df.iloc[0:5, 1:11].values.flatten().tolist()
# important_tags = []
# tags_list = []
# word_list = df.values.flatten().tolist()
# print(word_list)
# for tag in important_tags_raw:
#     tag_str = str(tag)
#     if '_' in tag_str:
#         important_tags.extend(tag_str.split('_'))
#     else:
#         important_tags.append(tag_str)
# important_tags = list(set(important_tags))
# print("i", important_tags)

# for tag in important_tags:
#     df2[tag] = 0

# df2 = df2.dropna(subset=['Token_Text'])

# for index, row in df2.iterrows():
#     token_text = row['Token_Text']
#     if isinstance(token_text, str): 
#         for tag in important_tags:
#             if tag in token_text:
#                 df2.at[index, tag] = 1
#             else:
#                 df2.at[index, tag] = 0


important_tags_raw = df.iloc[0:5, 1:11].values.flatten().tolist()
important_tags = []
topic_no = df.columns.tolist()
for tag in important_tags_raw:
    tag_str = str(tag)
    if '_' in tag_str:
        important_tags.extend(tag_str.split('_'))
    else:
        important_tags.append(tag_str)
important_tags = list(set(important_tags))
print("i", important_tags)

for topic in topic_no:
    print(topic)
    df2[topic] = 0

df2 = df2.dropna(subset=['Token_Text'])


for index, row in df2.iterrows():
    token_text = row['Token_Text']
    if isinstance(token_text, str): 
        for column in df.columns:
            if any(word in token_text for word in df[column]):
                df2.at[index, column] = 1
            else:
                df2.at[index, column] = 0


df2.head()
# for index, row in df2.iterrows():
#     token_text = row['Token_Text']
#     for column in df.columns[1:11]:
#         counter += 1
#         for index, word in df[column].items():
#             if isinstance(token_text, str): 
#                 if word in token_text:
#                     df2.at[index, counter] = 1
#                 else:
#                     df2.at[index, counter] = 0

# for index, row in df2.iterrows():
#     token_text = row['Token_Text']
#     presence = []
    # for column in df.columns:
    #     if any(word in token_text for word in df[column]):
#             presence.append(1)
#         else:
#             presence.append(0)
#     df2.loc[index, df.columns] = presence

# # Convert NaN values to 0
# df2.fillna(0, inplace=True)


# for column in df.columns[1:11]:
#     counter += 1
#     print(counter)
#     for index, word in df[column].items():
#         print(index, word)
#         if isinstance(word, str): 
#             if word in token_text:
#                 df2.at[index, counter] = 1
#             else:
#                 df2.at[index, tag] = 0



i ['Tick', 'Insert', 'height', 'answer', 'population', 'edge', 'use', 'growth', 'position', 'complete', 'map', 'earthquake', 'rainfall', 'Describe', 'Table', 'Study', 'two', 'point', 'Give', 'migration', 'industry', 'climate', 'change', 'rate', 'name', 'volcano', 'type', 'road', 'cross', 'water', 'land', 'Name', 'number', 'feature', 'shown', 'one', 'temperature', 'show', 'river', 'section', 'year', 'extract', 'country', 'area', 'question']
Topic #01
Topic #02
Topic #03
Topic #04
Topic #05
Topic #06
Topic #07
Topic #08
Topic #09
Topic #10


,Paper_ID,Question_ID,Question_Type,Question_Text,Token_Text,Word_Count,Topic #01,Topic #02,Topic #03,Topic #04,Topic #05,Topic #06,Topic #07,Topic #08,Topic #09,Topic #10
0,0460_m17_qp_22.pdf,1,MAIN,"Study the map extract for Ballyvaghan, Ireland...",Study map extract Ballyvaghan Ireland scale 1:50,71,0,1,0,0,0,1,0,1,1,0
1,0460_m17_qp_22.pdf,(a),SUB,The map has blue grid lines which make square...,map blue grid line make square area land one g...,126,1,1,0,1,1,1,1,1,1,1
5,0460_m17_qp_22.pdf,(b),SUB,(i) Height above sea level is shown by contou...,Height sea level shown contour line difference...,159,1,1,1,1,1,1,1,1,1,1
6,0460_m17_qp_22.pdf,(ii),SUB,What does the green shading in grid square 28...,green shading grid square show,55,0,1,0,0,0,1,1,1,1,0
7,0460_m17_qp_22.pdf,(c),SUB,Fig. 1 shows some of the features in the east...,show feature east map extract Study map extrac...,140,0,1,0,1,1,1,1,1,1,0


In [198]:
column_names = df2.columns.tolist()
print(column_names)


['Paper_ID', 'Question_ID', 'Question_Type', 'Question_Text', 'Token_Text', 'Word_Count', 'Topic #01', 'Topic #02', 'Topic #03', 'Topic #04', 'Topic #05', 'Topic #06', 'Topic #07', 'Topic #08', 'Topic #09', 'Topic #10']


In [199]:
df2.dtypes

Paper_ID         object
Question_ID      object
Question_Type    object
Question_Text    object
Token_Text       object
Word_Count        int64
Topic #01         int64
Topic #02         int64
Topic #03         int64
Topic #04         int64
Topic #05         int64
Topic #06         int64
Topic #07         int64
Topic #08         int64
Topic #09         int64
Topic #10         int64
dtype: object

In [200]:
df2['Question_Text'].apply(lambda x:nt.TextFrame(x).noise_scan())

0       {'text_noise': 11.267605633802818, 'text_lengt...
1       {'text_noise': 11.11111111111111, 'text_length...
5       {'text_noise': 11.949685534591195, 'text_lengt...
6       {'text_noise': 9.090909090909092, 'text_length...
7       {'text_noise': 12.142857142857142, 'text_lengt...
8       {'text_noise': 4.761904761904762, 'text_length...
9       {'text_noise': 12.121212121212121, 'text_lengt...
10      {'text_noise': 11.11111111111111, 'text_length...
11      {'text_noise': 12.068965517241379, 'text_lengt...
12      {'text_noise': 15.714285714285714, 'text_lengt...
13      {'text_noise': 5.755395683453238, 'text_length...
14      {'text_noise': 10.16949152542373, 'text_length...
15      {'text_noise': 11.805555555555555, 'text_lengt...
16      {'text_noise': 6.338028169014084, 'text_length...
17      {'text_noise': 11.881188118811881, 'text_lengt...
18      {'text_noise': 15.11627906976744, 'text_length...
19      {'text_noise': 15.517241379310345, 'text_lengt...
20      {'text

In [201]:
df2['Question_Text'].apply(lambda x:nt.TextExtractor(x).extract_stopwords())

0                                     [the, for, the, is]
1       [the, has, which, make, what, of, does, one, one]
5       [above, is, by, what, is, the, in, between, th...
6                                   [what, does, the, in]
7       [some, of, the, in, the, of, the, and, the, an...
8                                                      []
9                                           [the, of, at]
10                               [give, the, of, the, at]
11                        [give, the, above, of, the, at]
12          [the, of, the, between, d, and, between, and]
13                                [the, of, the, at, and]
14      [a, along, the, from, the, at, to, the, at, ho...
15      [how, much, further, does, the, along, the, th...
16      [which, of, the, the, that, the, from, to, one...
17                    [is, a, through, the, of, from, to]
18                    [on, a, to, show, the, of, the, at]
19                                  [using, the, the, on]
20            

In [202]:
df2['Question_Text'].apply(nfx.remove_stopwords)

0       Study map extract Ballyvaghan, Ireland. scale ...
1       map blue grid lines squares. area land grid sq...
5       (i) Height sea level shown contour lines. diff...
6                    green shading grid square 2807 show?
7       Fig. 1 shows features east map extract. Study ...
8                                     Identify feature A.
9                                   Identify type road B.
10             six-figure grid reference road junction C.
11                        height sea level spot height D.
12                        Describe gradient road: E; E C.
13      Fig. 2 shows location coastal settlements Ball...
14      person travels N67 road main road junction Bal...
15      person travel N67 road straight line distance ...
16      following statements describes directions pers...
17      Fig. 3 cross section summit (top) Moneen Mount...
18      Fig. 3, use labelled arrow position lake Lough...
19                map, complete cross section Fig. 3. [2]
20      map ev

In [203]:
corpus = df2['Question_Text'].apply(nfx.remove_stopwords)

In [204]:
tfidf = TfidfVectorizer()
Xfeatures = tfidf.fit_transform(corpus).toarray()
Xfeatures

array([[0.       , 0.3187649, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [206]:
# y = df2[['settlement', 'type', 'rate', 'answer', 'question', 'temperature', 'population', 'feature', 'Table', 'road', 'Using', 'earthquake', 'migration', 'cross_section', 'number', 'show', 'area', 'shown', 'show', 'edge_map', 'Suggest', 'land_use', 'population', 'volcano', 'name', 'rainfall', 'country', 'map_extract', 'Study', 'point', 'reason', 'Name', 'growth', 'complete', 'use', 'change', 'two', 'Describe', 'water', 'Tick_one', 'one', 'industry', 'Give', 'height', 'Insert', 'climate', 'year', 'river', 'position', 'Tick', 'evidence', 'show', 'level', 'line', 'answer', 'place', 'show', 'area', 'South', 'railway', 'area', 'plate', 'height_sea', 'shown', 'page', 'shown', 'box', 'show', 'Africa', 'Measure', 'town', 'plate_boundary', 'spot_height', 'island', 'mark', 'instrument', 'Tick', 'extract', 'Identify', 'south', 'suggest', 'Plate', 'region', 'Circle', 'paper', 'mm', 'table', 'Using', 'area', 'north', 'pattern', 'road', 'farming', 'Give', 'Write', 'France', 'one', 'Explain', 'two', 'bearing', 'three', 'scale', 'reference', 'Using_information', 'diagram', 'weather', 'world', 'answer_question', 'shown', 'metre_metre', 'two', 'Ocean', 'six-figure_grid', 'level', 'space', 'Study_map', 'people', 'settlement', 'Insert_show', 'sea_level', 'development', 'intensity', 'Sea', 'building', 'end', 'extract', 'per', 'Study_map', 'settlement', 'along', 'electricity', 'UK', 'State', 'metre', 'food', 'month', 'tick', 'identify_following', 'describe', 'Find', 'country', 'Pacific', 'Mediterranean', 'contour', 'graph', 'USA', 'percentage', 'relief', 'use', 'correct_answer']]
# print(y)
y=df2[['Topic #01', 'Topic #02', 'Topic #03', 'Topic #04', 'Topic #05', 'Topic #06', 'Topic #07', 'Topic #08', 'Topic #09', 'Topic #10']]
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)
binary_rel_clf = BinaryRelevance(MultinomialNB())
binary_rel_clf.fit(X_train,y_train)

BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

In [207]:
br_prediction = binary_rel_clf.predict(X_test)
br_prediction.toarray()

array([[0, 0, 0, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0],
       ...,
       [0, 1, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [1, 1, 0, ..., 1, 1, 0]], dtype=int64)

In [208]:
accuracy_score(y_test,br_prediction)

0.25125

In [209]:
hamming_loss(y_test,br_prediction)

0.183375

In [210]:
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    clf_predictions = clf.predict(xtest)
    acc = accuracy_score(ytest,clf_predictions)
    ham = hamming_loss(ytest,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham}
    return result

In [211]:
clf_chain_model = build_model(MultinomialNB(),ClassifierChain,X_train,y_train,X_test,y_test)

In [212]:
clf_chain_model

{'accuracy:': 0.235, 'hamming_score': 0.20225}

In [213]:
clf_labelP_model = build_model(MultinomialNB(),LabelPowerset,X_train,y_train,X_test,y_test)

In [214]:
clf_labelP_model

{'accuracy:': 0.25, 'hamming_score': 0.26625}

In [215]:
ex1 = df2['Question_Text'].iloc[0]

In [216]:
vec_example = tfidf.transform([ex1])

In [217]:
binary_rel_clf.predict(vec_example).toarray()

array([[0, 1, 0, 0, 0, 1, 1, 1, 1, 0]], dtype=int64)

In [219]:
titles_to_predict = df2['Question_Text'].iloc[1:11]

vec_examples = tfidf.transform(titles_to_predict)

predictions = binary_rel_clf.predict(vec_examples).toarray()

print(predictions)




[[1 0 0 1 1 0 1 1 1 1]
 [0 0 1 1 0 0 1 0 0 0]
 [0 1 0 0 0 1 1 1 1 0]
 [1 1 0 1 0 1 1 1 1 0]
 [0 0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 0 1 1 0]
 [0 1 1 1 0 0 0 0 0 1]
 [0 0 1 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 1 1 0]
 [1 1 0 0 0 1 1 1 1 0]]


In [225]:
last_10_cols_df = df2.iloc[:, -10:]
array = np.array([1, 0, 0, 1, 1, 0, 1, 1, 1, 1])

matching_rows = df2[(last_10_cols_df == array).all(axis=1)]

print("Matching rows:")
print(matching_rows)

Matching rows:
                Paper_ID Question_ID Question_Type  \
543   0460_s15_qp_22.pdf         10           MAIN   
558   0460_s15_qp_23.pdf         (c)           SUB   
1568  0460_w14_qp_22.pdf         (c)           SUB   

                                          Question_Text  \
543   representing one of the island’s main attracti...   
558    Study the two named rivers in the centre of t...   
1568   Several countries along the Mekong River are ...   

                                             Token_Text  Word_Count  \
543   representing one island main attraction Montse...         410   
558   Study two named river centre area Mwewe Babuli...         380   
1568  Several country along Mekong River building la...         212   

      Topic #01  Topic #02  Topic #03  Topic #04  Topic #05  Topic #06  \
543           1          0          0          1          1          0   
558           1          0          0          1          1          0   
1568          1         